# Script to Extract Data
**Input Data:** Original Data  
**Output Data:** Data for a specific variable and spatial extent  
**Description:** Extracts data for a specific variable and spatial extent and exports them to a new file.  
**Date:** June 2022  
**Author:** Emma Perkins  
**Updated:** April 2023 by Teagan King  
**Note:** It is recommended to run this notebook with 50GB memory allocation.

In [1]:
# import relevant packages
import xarray as xr
import glob

In [2]:
# parameters for dask optimization
nworkers = 20
chunk_time = 73
chunk_lat = 100
chunk_lon = 100
nmem='30'

In [3]:
# Import dask
import dask

# Use dask jobqueue
from dask_jobqueue import PBSCluster

# Import a client
from dask.distributed import Client

# Setup your PBSCluster
cluster = PBSCluster(
    cores=1, # The number of cores you want
    memory=nmem+'GiB', # Amount of memory
    processes=1, # How many processes
    queue='casper', # The type of queue to utilize (/glade/u/apps/dav/opt/usr/bin/execcasper)
    local_directory='/glade/scratch/$USER/local_dask', # Use your local directory
    resource_spec='select=1:ncpus=1:mem='+nmem+'GB', # Specify resources
    account='P93300065', # Input your project ID here, previously this was known as 'project', now is 'account'
    walltime='04:00:00', # Amount of wall time
    interface='ib0', # Interface to use
)

# Scale up
cluster.scale(nworkers)

# Change your url to the dask dashboard so you can see it
dask.config.set({'distributed.dashboard.link':'https://jupyterhub.hpc.ucar.edu/stable/user/{USER}/proxy/{port}/status'})

# Setup your client
client = Client(cluster)

In [4]:
client

Connection method: Cluster object,Cluster type: dask_jobqueue.PBSCluster
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/tking/proxy/8787/status,
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/tking/proxy/8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.12.206.35:39790,Workers: 0
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/tking/proxy/8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


### Load Full Data

In [5]:
# full data
paths = '/glade/campaign/cesm/collections/cesmLE/CESM-CAM5-BGC-LE/lnd/proc/tseries/daily/H2OSNO/'  # change to your paths
file1 = paths+'b.e11.B1850C5CN.f09_g16.005.clm2.h1.H2OSNO.19000101-19991231.nc'
file2 = paths+'b.e11.B1850C5CN.f09_g16.005.clm2.h1.H2OSNO.20000101-20991231.nc'
# files = sorted(glob.glob(paths+names))
files = [file1, file2]
full_data = xr.open_mfdataset(files, concat_dim=None, parallel=True, chunks={"time":chunk_time, "lat":chunk_lat, "lon":chunk_lon})


In [6]:
full_data

<xarray.Dataset>
Dimensions:       (levgrnd: 15, levlak: 10, time: 73000, hist_interval: 2,
                   lon: 288, lat: 192)
Coordinates:
  * levgrnd       (levgrnd) float32 0.007101 0.02792 0.06226 ... 21.33 35.18
  * levlak        (levlak) float32 0.05 0.6 2.1 4.6 ... 18.6 25.6 34.33 44.78
  * time          (time) object 1900-01-02 00:00:00 ... 2100-01-01 00:00:00
  * lon           (lon) float32 0.0 1.25 2.5 3.75 ... 355.0 356.2 357.5 358.8
  * lat           (lat) float32 -90.0 -89.06 -88.12 -87.17 ... 88.12 89.06 90.0
Dimensions without coordinates: hist_interval
Data variables: (12/14)
    mcdate        (time) int32 dask.array<chunksize=(73,), meta=np.ndarray>
    mcsec         (time) int32 dask.array<chunksize=(73,), meta=np.ndarray>
    mdcur         (time) int32 dask.array<chunksize=(73,), meta=np.ndarray>
    mscur         (time) int32 dask.array<chunksize=(73,), meta=np.ndarray>
    nstep         (time) int32 dask.array<chunksize=(73,), meta=np.ndarray>
    time_bounds   (time, hist_interval) object dask.array<chunksize=(73, 2), meta=np.ndarray>
    ...            ...
    area          (time, lat, lon) float32 dask.array<chunksize=(36500, 100, 100), meta=np.ndarray>
    topo          (time, lat, lon) float32 dask.array<chunksize=(36500, 100, 100), meta=np.ndarray>
    landfrac      (time, lat, lon) float32 dask.array<chunksize=(36500, 100, 100), meta=np.ndarray>
    landmask      (time, lat, lon) float64 dask.array<chunksize=(36500, 100, 100), meta=np.ndarray>
    pftmask       (time, lat, lon) float64 dask.array<chunksize=(36500, 100, 100), meta=np.ndarray>
    H2OSNO        (time, lat, lon) float32 dask.array<chunksize=(73, 100, 100), meta=np.ndarray>
Attributes: (12/16)
    title:                                CLM History file information
    comment:                              NOTE: None of the variables are wei...
    Conventions:                          CF-1.0
    history:                              created on 10/06/14 12:01:31
    source:                               Community Land Model CLM4.0
    hostname:                             yellowstone
    ...                                   ...
    case_id:                              b.e11.B1850C5CN.f09_g16.005
    Surface_dataset:                      surfdata_0.9x1.25_simyr1850_c110921.nc
    Initial_conditions_dataset:           b40.1850.track1.1deg.006.clm2.r.086...
    PFT_physiological_constants_dataset:  pft-physiology.c110425.nc
    Time_constant_3Dvars_filename:        ./b.e11.B1850C5CN.f09_g16.005.clm2....
    Time_constant_3Dvars:                 ZSOI:DZSOI:WATSAT:SUCSAT:BSW:HKSAT

### Select Variable of Interest

In [7]:
data_var = 'H2OSNO'  # change to variable of interest from climate model data
data_select = full_data[data_var]

### Select Area of Interest

In [8]:
%%time

# determine variable names
lat_var = 'lat'  # name of latitude variable for original data
lon_var = 'lon'  # name of longitude variable for original data

# rename lat lon variables to all be lat lon
data_select = data_select.rename({lat_var: 'lat', lon_var: 'lon'})

# sort by latitude:
data_select = data_select.sortby('lat')

lon_type = 'long3'  # longitude coordinate type: long1 (-180 - 180) or long 3 (0 - 360)
if lon_type == 'long3':
    lon_new = (data_select.lon + 180) % 360 - 180
    data_select['lon'] = lon_new
data_select = data_select.sortby('lon')

# select input area from left to right / west to east:
lat_min = 50  # minimum latitude
lat_max = 90  # maximum latitude
lon_min = 150  # minimum longitude
lon_max = -100  # maximum longitude

if lon_min < lon_max:
    data_select = data_select.sel(lat=slice(lat_min, lat_max), lon=slice(lon_min, lon_max))
else:
    data_select1 = data_select.sel(lat=slice(lat_min, lat_max), lon=slice(lon_min, 180))
    data_select2 = data_select.sel(lat=slice(lat_min, lat_max), lon=slice(-180, lon_max))
    data_select = xr.concat([data_select1, data_select2], dim='lon')
data_select = data_select.sortby('lon')

<timed exec>:6: UserWarning: rename 'lat' to 'lat' does not create an index anymore. Try using swap_dims instead or use set_index after rename to create an indexed coordinate.
<timed exec>:6: UserWarning: rename 'lon' to 'lon' does not create an index anymore. Try using swap_dims instead or use set_index after rename to create an indexed coordinate.
/glade/u/home/tking/.conda/envs/cpad/lib/python3.10/site-packages/xarray/core/indexing.py:1374: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]


CPU times: user 350 ms, sys: 3.39 ms, total: 354 ms
Wall time: 360 ms


/glade/u/home/tking/.conda/envs/cpad/lib/python3.10/site-packages/xarray/core/indexing.py:1374: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]


### Standardize Time Step
Can skip if already using the desired time step.

In [9]:
%%time

analysis_time_type = '1D'  # time step for analysis (ex: 6H, 1D, 1M, 1Y, etc.)

# choose either .sum() or .mean() for accumulation or instantaneous variables respectively
data_select = data_select.resample(time=analysis_time_type).mean('time')

CPU times: user 1min 58s, sys: 4.44 s, total: 2min 3s
Wall time: 2min 10s


### Export Data (~15 minutes)

In [10]:
data_select

<xarray.DataArray 'H2OSNO' (time: 73000, lat: 43, lon: 89)>
dask.array<stack, shape=(73000, 43, 89), dtype=float32, chunksize=(1, 43, 56), chunktype=numpy.ndarray>
Coordinates:
  * lon      (lon) float32 -180.0 -178.8 -177.5 -176.2 ... 176.2 177.5 178.8
  * lat      (lat) float32 50.42 51.36 52.3 53.25 ... 87.17 88.12 89.06 90.0
  * time     (time) object 1900-01-02 00:00:00 ... 2100-01-01 00:00:00
Attributes:
    long_name:     snow depth (liquid water)
    units:         mm
    cell_methods:  time: mean

In [11]:
%%time

outpath = '/glade/scratch/tking/cesm/'  # path for new observational data file
data_name = 'cesmLE_B1850C5CN_H2OSNO_1900_2099_1D_MRBplus'  # name for new data file

data_select.load().to_netcdf(outpath+data_name+'.nc')

CPU times: user 13min 8s, sys: 18.8 s, total: 13min 26s
Wall time: 14min 3s
